**NLP - Aula 3 - Similaridade Textual**
===============================

Durante a prática de hoje vamos testar alguns dos conceitos que vimos na aula teórica sobre similaridade textual

## Similaridade de Jaccard ##

Vamos realizar o teste criando o algoritmo a partir da fórmula apresentada na aula J(X,Y) = |X∩Y| / |X∪Y| e um exemplo com a biblioteca __nltk.metrics.distance__

Comparem as duas frases exemplo:


*   AI is our friend and it has been friendly
*   AI and humans have always been friendly



In [3]:
#Inclua aqui o seu próprio algoritmo para calcular a distância de Jaccard
import nltk


str1 = "AI is our friend and it has been friendly"
str2 = "AI and humans have always been friendly"

tk1 = str1.split()
tk2 = str2.split()

print(tk1)
print(tk2)

a = set(tk1)
b = set(tk2)

jaccard = len(a.intersection(b)) / len(a.union(b))

print(jaccard)


['AI', 'is', 'our', 'friend', 'and', 'it', 'has', 'been', 'friendly']
['AI', 'and', 'humans', 'have', 'always', 'been', 'friendly']
0.3333333333333333


In [10]:
#Inclua aqui o resultado utilizando a biblioteca já criada

from nltk.metrics.distance import jaccard_distance

jd = jaccard_distance(a,b)
similarity = 1 - jd

print(similarity)
                      

0.33333333333333337


## Plágio de texto

Através do cálculo da similaridade do Cosseno, com abordagem TFIDF + Bag of Words, iremos comparar 5 artigos de notícias e observarmos o percentual de similaridade que eles tem entre si e com base nisso analisarmos se houve cópia de uma notícia entre um site ou outro.

A mesma abordagem poderia ser utilizada para várias outras aplicações de plágio, tais como a originalidade de um artigo, cópias entre exercícios de alunos, etc.

As urls dos artigos que iremos analisar são:

*  https://drive.google.com/uc?export=download&id=191Ae2usY5cEAPJvgxKpIve6Is1sqf5Lh
*  https://drive.google.com/uc?export=download&id=1oIbaR3uYZajpFfOQlY2-Jfs_pik-YfxY
*  https://drive.google.com/uc?export=download&id=1vSzS0ZwxN2KQjOZo4QgIIVsgtEbhtmXQ
*  https://drive.google.com/uc?export=download&id=1Xou8jQIOk7GomEFtZmqcII_6vyiPo4Di
*  https://drive.google.com/uc?export=download&id=1U1Le1LWAQbrclqL-oIt65T9PtXsqW1hN


Primeira parte do nosso código será a importação de todas as bibliotecas que iremos utilizar:

Para funções gerais de limpeza e tokenização
*  NLTK
*  string

Para a leitura do conteúdo _raw_ das urls dos artigos
*  requests

Para calcular TF-IDF e a similaridade do cosseno
*  from sklearn.feature_extraction.text import TfidfVectorizer
*  from sklearn.metrics.pairwise import cosine_similarity


In [13]:
import nltk
import string
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Recuperação e tratamento dos textos ###

Como primeira parte do nosso desenvolvimento criaremos uma função com o nome **RetrieveAndProcessNews** que vai recuperar os conteúdos dos artigos e tratar os textos com os seguintes pré-processamentos:
*  Tokenizar as palavras
*  Remover stopwords
*  Executar o processo de steeming (PorterStemmer)
*  Remover pontuação
*  Converter o texto para letras minúsculas

In [32]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

urls = ['https://drive.google.com/uc?export=download&id=191Ae2usY5cEAPJvgxKpIve6Is1sqf5Lh', \
    'https://drive.google.com/uc?export=download&id=1oIbaR3uYZajpFfOQlY2-Jfs_pik-YfxY', \
    'https://drive.google.com/uc?export=download&id=1vSzS0ZwxN2KQjOZo4QgIIVsgtEbhtmXQ', \
    'https://drive.google.com/uc?export=download&id=1Xou8jQIOk7GomEFtZmqcII_6vyiPo4Di', \
    'https://drive.google.com/uc?export=download&id=1U1Le1LWAQbrclqL-oIt65T9PtXsqW1hN']

def RetrieveAndProcessNews(urls):
    processed_texts = []
    stopwords = nltk.corpus.stopwords.words('english')
    stemmer = PorterStemmer()
    
    for url in urls:
        raw_text = requests.get(url).text
        
        text_cleaned = raw_text.translate(str.maketrans('','',string.punctuation)).lower()
        
        tokens = word_tokenize(text_cleaned)
        
        words = [stemmer.stem(word) for word in tokens if word not in stopwords]

        result = ' '.join(words)
        processed_texts.append(result)
        
    return processed_texts



#for ct in clean_texts:
#    print('[%s]\n'%(ct))



## TF-IDF e Similaridade do Cosseno ##

Vamos criar uma função para calcular TF-IDF e em seguida a função para calcular a similaridade do cosseno


In [28]:
def CalculateTFIDF(corpus):
    tfidf = TfidfVectorizer()
    x = tfidf.fit_transform(corpus)
    tfs_Values = x.toarray()
    tfs_Term = tfidf.get_feature_names()
    
    return tfs_Values, tfs_Term

In [67]:
def CalculateAndPrintCosine(tfs_Values, filenames):
    for f in filenames:
        print(f, end='  ')
    print()
        
    for i in range(len(filenames)):
        print(filenames[i], end=' ')
        for j in range(len(filenames)):
            matrixValue = cosine_similarity([tfs_Values[i]],[tfs_Values[j]])
            numValue = matrixValue[0][0]
            print(numValue, end='  ')
        print()
            
    

In [50]:
processed_texts = RetrieveAndProcessNews(urls)
tfs_Values, tfs_Term = CalculateTFIDF(processed_texts)

print(tfs_Values)
print()
print(tfs_Term)


[[0.0606459  0.         0.03032295 ... 0.03032295 0.15305173 0.        ]
 [0.07638777 0.         0.03819388 ... 0.03819388 0.0321299  0.        ]
 [0.07958498 0.         0.03979249 ... 0.03979249 0.0334747  0.        ]
 [0.         0.0462642  0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.04385333 0.03891968]]

['10', '100', '1040', '1135', '130', '136', '141', '15', '150', '156', '16', '160', '16260', '16270', '16393', '185', '1994', '1997', '200', '2003', '2004', '225', '23', '2392', '24', '30', '350', '37', '39', '400', '455', '51', '545', '58', '82', '87', 'abil', 'abl', 'ac', 'accept', 'account', 'achiev', 'ad', 'admit', 'afford', 'agre', 'ahead', 'alastair', 'alleg', 'allow', 'alreadi', 'although', 'among', 'amount', 'analyst', 'anglofrench', 'angri', 'announc', 'annual', 'anoth', 'anyth', 'appeal', 'appel', 'applic', 'approv', 'april', 'argument', 'around', 'arrang', 'arriv', 'ask', 'asset', 'assist', 'attack', 'august', 'aut

##  Hora de criar a função Main ##





In [68]:
CalculateAndPrintCosine(tfs_Values, ['f1','f2','f3','f4','f5'])

f1  f2  f3  f4  f5  
f1 0.9999999999999999  0.7448447086216057  0.7684204247210125  0.17461294608359224  0.11780133878807081  
f2 0.7448447086216057  0.9999999999999991  0.976346189554524  0.12784277969773092  0.09133686524835465  
f3 0.7684204247210125  0.976346189554524  1.0000000000000004  0.12933443080197535  0.09724710251455776  
f4 0.17461294608359224  0.12784277969773092  0.12933443080197535  1.0000000000000007  0.07127581415325  
f5 0.11780133878807081  0.09133686524835465  0.09724710251455776  0.07127581415325  1.0000000000000007  
